# Imports

Llevamos a cabo los imports necesarios para realizar los ejercicios de esta sección

In [ ]:
!pip install h2o

In [ ]:
import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt
import seaborn as sns
import h2o
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import IsolationForest
from sklearn.datasets import make_moons
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

# Configuraciones

Definimos una serie de configuraciones que nos sirvan para todo el notebook

In [ ]:
np.random.seed(1993)
%matplotlib inline

# Métodos paramétricos

Para el estudio de estas técnicas de detección de outliers, vamos a emplear un conjunto de datos generados aleatoriamente, que cumplen con la premisa que marcan los métodos parametricos dee que los datos siguen una distribución normal

In [ ]:
numero_variables=4
df_normal = pd.DataFrame(np.random.normal(scale=10.0, 
                                        size=(1000, numero_variables)),
                           columns=['variable_{}'.format(i) for i in range(numero_variables)])
df_normal.head()

Ahora vamos a mirar a sus histoigramas, podemos observar como efectivamente, siguen una distribución normal

In [ ]:
df_normal.hist(figsize=(10,10));

## Identificación de outliers en nuestros datos

Los metodos de detección de outliers parametricas más extendidos son:

    - Detección empleando la desviación estandar.
    - Detección empleando el rango interquantil.
    
Vamos a generar dos funciones que nos permitan calcular estos valores y aplicarlos sobre nuestras variables

### Metodo de la desviacion estandar

In [ ]:
def fuera_std(s, nstd=3.0, return_thresholds=False):
    """
    Definir una funcion que devuelva si el punto de la variable se 
    encuentra dentro o fuera de la franja de desviación estandar 
    de la distribución de la variables.
    Si return_thresholds es True se le pedirá que nos mande los 
    umbrales en los que está haciendo el corte
    """
    ###

In [ ]:
outliers_std = fuera_std(df_normal.variable_0,3,False)

In [ ]:
plt.figure(figsize=(8,6))
sns.distplot(df_normal['variable_0'], kde=False);
plt.vlines(df_normal['variable_0'][outliers_std], 
           ymin=0, ymax=110, 
           linestyles='dashed');

### Metodo del rango interquartil

In [ ]:
def fuera_iqr(s, 
              k=1.5, 
              return_thresholds=False):
    """
   Calcula el IQR y lo multiplica por un k que define el 
   limite final para detectar outliers.
    Si return_thresholds es True se le pedirá que nos mande los 
    umbrales en los que está haciendo el corte
   
    """
    # calcular IQR
    q25, q75 = ###
    iqr = ###
    # calcular el corte de los outliers
    cut_off = ###
    limite_inferior, limite_superior = ###
    if return_thresholds:
        return limite_inferior, limite_superior
    else: 
        return [True if x < limite_inferior or x > limite_superior else False for x in s]

In [ ]:
outliers_iqr = fuera_iqr(df_normal.variable_0,1.5,False)

In [ ]:
plt.figure(figsize=(8,6))
sns.distplot(df_normal['variable_0'], kde=False);
plt.vlines(df_normal['variable_0'][outliers_iqr], 
           ymin=0, ymax=110, 
           linestyles='dashed');

### Evolución del umbral en función de los multiplicadores que se atribuyan

Vamos a comprobar como varia el umbral con el que se corta a la distribución normal en función del multiplicador que se asocie a la desviación estandar o al rango interquantil

In [ ]:
def plot_umbrales_std(dataframe, col, nstd=2.0, color='red'):
    """
    Plot de los limites de umbrales
    """
    inferior, superior = fuera_std(dataframe[col], nstd=nstd, return_thresholds=True)
    plt.axvspan(min(dataframe[col][dataframe[col] < inferior], 
                    default=dataframe[col].min()), 
                inferior, 
                alpha=0.2, 
                color=color);
    
    plt.axvspan(superior, max(dataframe[col][dataframe[col] > superior],
                           default=dataframe[col].max()), 
                alpha=0.2, 
                color=color);

In [ ]:
column = 'variable_0'
sns.distplot(df_normal[column], kde=False)
plot_umbrales_std(df_normal, column, nstd=2.0, color='red');
plot_umbrales_std(df_normal, column, nstd=3.0, color='blue');
plot_umbrales_std(df_normal, column, nstd=4.0, color='yellow');

In [ ]:
def plot_umbrales_iqr(dataframe, col, k=1.5, color='red'):
    """
    Plot de los limites de umbrales
    """
    inferior, superior = fuera_iqr(dataframe[col], k=k, return_thresholds=True)
    
    plt.axvspan(min(dataframe[col][dataframe[col] < inferior], 
                    default=dataframe[col].min()), 
                inferior, 
                alpha=0.2, 
                color=color);
    
    plt.axvspan(superior, max(dataframe[col][dataframe[col] > superior],
                           default=dataframe[col].max()), 
                alpha=0.2, 
                color=color);

In [ ]:
column = 'variable_0'
sns.distplot(df_normal[column], kde=False)
plot_umbrales_iqr(df_normal, column, k=1.5, color='red');
plot_umbrales_iqr(df_normal, column, k=2.0, color='blue');
plot_umbrales_iqr(df_normal, column, k=3.0, color='yellow');

Podemos observar ocmo el IQR es mucho mas restrictivo que el metodo de la desviación

# Elaboración de gráficos Boxplot

## Lectura de los datos

Para el ejercicio de dibujar y trabajar ocn boxplot, emplearemos los datos de un dataset real que recoge cierta información sobre los coches que se están vendiendo en un portal web. este dataset recoge la sigueinte información:

**- car:** marca del coche

**- price:** precio de venta del anunciante (en dolares

**- body:** tipo de coche

**- mileage:** km que tiene el coche

**- engV:** rounded engine volume (‘000 cubic cm)

**- engType:** tipo de motor

**- registration:** si el coche esta registrado a nivel nacional.

**- year:** año de fabricación

**- model:** modelo especifico de coche

**- drive:** tipo de traccion


In [ ]:
df_coches = pd.read_csv("https://raw.githubusercontent.com/jguijarh/The_Valley_outliers_and_residuals/main/anuncios-coche/car_ad.csv", sep=',', encoding='latin-1')
df_coches.head()

A simple vista podemos ver como hay valores nulos y otros problemas que deberían solucionarse en una fase precia de EDA. Para no alargar el ejercicio ignoraremos estos pasos dado que no impactan directamente en lo que queremos realizar.

### Calcular la antiguedad y los kilometros anuales 

Lo primero que vamos a realizar en este ejercicio es calcular dos variables que pueden ser interesantes:

    - La antiguedad de los coches 
    - Las millas por año que han recorrido.
    
Eliminamos los coches nuevos para no meter ruido en los analisis futuros.

In [1]:
df_coches['antiguedad'] = ###
df_coches['mileage_anio'] = ###
df_coches = ###

SyntaxError: ignored

## Dibujar boxplot

con los datos ya dispuestos, vamos a representar ciertas variables categoricas en función de otras numericas.

#### Boxplot de tipo de motor vs antiguedad del vehiculo
Llevar a cabo la representacion de boxplot teniendo en cuenta la variable tipo de motor y observar la antiguedad.

Usaremos para este cometido la función boxplot de seaborn 

In [ ]:
plt.figure(figsize=(16, 10))
###

Analicemos los resultados.

### Millas anuales vs antiguedad del vehículo
Llevar a cabo la representacion de boxplot teniendo en cuenta la agrupacion por la antiguedad de los coches

In [ ]:
plt.figure(figsize=(16, 10))
###

Analizamos los resultados

#### Obtengamos la evolución de las millas recorridas en un año (mediana)

In [ ]:
df_anio_mileage = ###
df_anio_mileage.head()

Lo dibujamos

In [ ]:
plt.figure(figsize=(16, 10))
sns.lineplot(df_anio_mileage.antiguedad, df_anio_mileage.mileage_anio)

Podemos observar ocmo hay ciertos tramos que tienen un comportamiento similar, que nos pueden permitir transformar nuestra variable continua antiguedad, en una varable por tramos que nos facilite el estudio del boxplot.

In [ ]:
condiciones = [df_coches.antiguedad<=3, 
              (df_coches.antiguedad>3) & (df_coches.antiguedad<=6), 
              (df_coches.antiguedad>6) & (df_coches.antiguedad<=10),
             (df_coches.antiguedad>10) & (df_coches.antiguedad<=15),
             (df_coches.antiguedad>15) & (df_coches.antiguedad<=20),
              (df_coches.antiguedad>20) & (df_coches.antiguedad<=30),
             df_coches.antiguedad > 30]

intervalo = ['menos_de_3', 'entre_3_y_6','entre_6_y_10',
            'entre_10_y_15','entre_15_y_20','entre_20_y_30',
            'mas_de_30']

df_coches['antiguedad_group'] = np.select(condiciones, intervalo)

Volvemos a realizar el boxplot pero esta vez contra la nueva variable agrupada

In [ ]:
plt.figure(figsize=(16, 10))
###

Analicemos el gráfico

# DBScan

Vamos a ver el funcionamiento de DBScan con dos tipos de datasets:

    - Un dataset de "juguete" para observar como se comporta el algoritmo.
    - Sobre un dataset y variable de un problema real, que es el de los precios de las casas de una ciudad, comparando sus metros cuadrados contra su precio

Generamos datos aleatorios con make_moons.

In [ ]:
X, y = make_moons(n_samples=400, noise=0.05, random_state=0)
x = X[:, 0]
y = X[:, 1]
sns.scatterplot(x, y, legend = False);

Definimos un DBSCan con sklearn y predecimos los clusters

In [ ]:
model = ###
clusters = ###

Nuestro modelo a identificado lo siguiente:

In [ ]:
sns.scatterplot(x, y, hue = clusters);

Podemos jugar con el ruido que le metemos al modelo y con la parametrización del modelo DBSCAN para forzar situaciones con este ejemplo sencillo

### DBScan sobre los datos de pisos de Melbourne

Empleamos un dataset de casas de melbourne para examinar como identifica DBScan anomalias dentro de la relación entre dos de sus variables.

Este es un problema tipico de kaggle y estamos analizando la variable target del problema 'SalePrice' respecto a 'GrLivArea', que es una de las variables más importantes cuando se modeliza este problema, pues es el area del inmueble.

#### Leemos los datos

In [ ]:
df_houses = pd.read_csv("https://raw.githubusercontent.com/jguijarh/The_Valley_outliers_and_residuals/main/precio-casas/train.csv")
df_houses.head()

Elaboramos una función que nos permite evaluar el rendimiento de DBScan en estos datos

In [ ]:
def dbscan_outliers(df, 
                    var1,var2,
                    dbscan_eps,
                    dbscan_minsample,
                    get_cluster_num):
    '''Funcion que define los datos escalados que se le pasan y busca outliers mediante DBScan en ellos'''
    plt.figure(figsize=(8,8))
    
    scaler = RobustScaler()
    scale_var1 = ###
    scale_var2 = ###
    # Generamos un dataframe con las nuevas variables
    df_temp = ###
    
    # Definimos el modelo DBScan
    clustering = ###
    
    #asignamos las predicciones
    df_temp["c"] = ###
    df_temp.index = ###
    
    # Visualizamos los resultados
    sns.scatterplot()###
    
    return df_temp[df_temp["c"]==get_cluster_num].index

In [ ]:
outlier_index = dbscan_outliers(df_houses,
                                "GrLivArea", "SalePrice",
                                dbscan_eps=.7,
                                dbscan_minsample=100,
                                get_cluster_num=-1)

print(f"Outliers identificados: {len(outlier_index)}")

# Isolation Forest

Finalmente vamos a ver la aplicaciónd e uno de los algoritmos más extendidos en la actualidad para la detección de outliers.

Esto es debido a su sencillez de implementar y a su eficacia,, como ya hemos visto durante la lección.

Para cambiar un poco vamos a realizar este ejercicio empleando la libreria H2O.

In [ ]:
h2o.init()

### Lectura de los datos

Vamos a leer los datos de los que dispone un departamenteo de Rrecursos humanos de sus trabjadores (ficticios) y vamos a emplear esta información para ver a que clase de trabajadores detecta como extraños nuestro isolation forest

In [ ]:
employee_data = h2o.import_file('https://raw.githubusercontent.com/jguijarh/The_Valley_outliers_and_residuals/main/HR/HR_info_empleados.csv')

### Definimos el modelo con H2o

La forma de definir el modelo es muy similar a sklearn

Seleccionamos las features a utilizar para el entrenamiento del modelo

In [ ]:
estimadores = ['Age', 'BusinessTravel', 
               'DistanceFromHome', 'Education', 
               'Gender', 'JobInvolvement', 'JobLevel', 
               'MaritalStatus', 'MonthlyIncome', 'NumCompaniesWorked', 
               'OverTime', 'PercentSalaryHike',
               'PerformanceRating', 'TotalWorkingYears', 
               'TrainingTimesLastYear', 'YearsAtCompany', 
               'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager']

In [ ]:
from h2o.estimators import H2OIsolationForestEstimator
isolation_model = H2OIsolationForestEstimator(model_id = "isolation_forest", 
                                              seed = 1993)
isolation_model.train(training_frame = employee_data, x = estimadores)

Lo que nos predice el modelo de h2o es el mena_length, que viene a significar como de profunda es la rama en la que se ha quedado ese registro en función de sus variables para realizar su segregación.

Cuanto menor sea este valor, menos divisiones serán necesarias para dividir este registro y por lo tanto más propenso a ser elegido como outlier será.

Podemos ver como esto queda verificado con el histograma de este valor, al haber menos valores con un valor pequeño de mean_length

In [ ]:
predicciones = isolation_model.predict(employee_data)
predicciones["mean_length"].hist()

A la vista de este histograma, podemos nostros mismos marcar un umbral en el cual identificar los outliers. Esto nos permite un poco más de versatilidad para parametrizar el modelo.

In [ ]:
umbral_outlier = 5.5
anomalias = employee_data[predictions["mean_length"] < umbral_outlier]
print("Numero de anomalias detectadas: " + str(anomalias.nrow))

Finalmente, calculamos añadimos este valor a nuestro dataframe de entrada al modelo.

In [ ]:
data_con_outliers = employee_data[:, :]
data_con_outliers["anomalia"] = (predicciones["mean_length"] < umbral_outlier).ifelse("Si", "No")
data_con_outliers["anomalia"].table()

UIna vez fuiinalizada la detección podemos convertirlo en pandas dataframe y seguir trabajando een por ejemplo representaciones gráficas de los datos.

In [ ]:
df_anomalias = h2o.as_list(data_con_outliers)

### Scatter plot de años que el empleado lleva en la empresa contra los años que lleva sin promocionar, coloreado por si el registro se considera outlier o no 

In [ ]:
plt.figure(figsize=(16, 10))
sns.scatterplot(x="TotalWorkingYears", y="YearsSinceLastPromotion", hue= 'anomalia',
            data=df_anomalias);

### Futuros pasos o lineas a investigar

1. Para el caso del isolation forest, construid un arbol, que tenga como variable target la anomalia y estudiad los cortes que hace el árbol para ver porque discrimina ciertas regiones de los datos.
2. Probar el algoritmo Isolation forest pero con sklearn y usad un volumen de datos más grande. Comparad tiempos.